In [ ]:
import os
import scipy.misc
import numpy as np
from utils import pp, visualize, show_all_variables
import tensorflow as tf
from Ano_Model_0625 import DCGAN
import time

#gpu setting
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"]="7"
run_config = tf.ConfigProto()
run_config.gpu_options.allow_growth=True

#arguments parsers
flags = tf.app.flags

#name, defulat value, description
#below are for train and test
flags.DEFINE_integer("epoch", 20, "Epoch to train [25]")
flags.DEFINE_integer("test_epoch", 100, "Epoch for latent mapping in anomaly detection to train [100]")
flags.DEFINE_float("d_learning_rate", 0.0002, "Learning rate of for adam of Discriminator [0.0002]")
flags.DEFINE_float("g_learning_rate", 0.0002, "Learning rate of for adam of Generator [0.0002]")

flags.DEFINE_float("beta1", 0.9, "Momentum term of adam [0.5]")
flags.DEFINE_float("test_learning_rate", 0.05, "Learning rate = Z_lr for finding latent variable z [0.05]")
flags.DEFINE_float("train_size", np.inf, "The size of train images [np.inf]")
flags.DEFINE_boolean("train", False, "True for training, False for testing [False]")
flags.DEFINE_boolean("anomaly_test", False, "True for anomaly test in test directory, not anomaly test [False]")
flags.DEFINE_boolean("visualize", False, "True for visualizing, False for nothing [False]")

#below are for model construction
flags.DEFINE_integer("batch_size", 64, "The size of batch images for GAN training[64]")
flags.DEFINE_integer("input_height", 64, "The size of image to use (will be center cropped). [108]")
flags.DEFINE_integer("input_width", None, "The size of image to use (will be center cropped). If None, same value with input_height [None]")
flags.DEFINE_integer("output_height", 64, "The size of the output images to produce [64]")
flags.DEFINE_integer("output_width", None, "The size of the output images to produce. ")

flags.DEFINE_string("input_fname_pattern", "*.jpg", "Glob pattern of filename of input images [*]")
flags.DEFINE_string("checkpoint_dir", "checkpoint", "Directory name to save the checkpoints [checkpoint]")
flags.DEFINE_string("sample_dir", "./samples", "Directory name to save the image samples [samples]")
flags.DEFINE_string("train_dir", "/home/sungmin2/py35_codes/00_AnoGAN/EEG_DATA/test4coderuntime/no/processed/", "Directory name to load training data")

# Anomaly detection
flags.DEFINE_integer("test_batch_size", 1, "The size of test batch images in anomaly detection to [1]")
flags.DEFINE_string("test_dir", "/home/sungmin2/py35_codes/00_AnoGAN/EEG_DATA/test4coderuntime/sz/processed/", "Directory name to load the anomaly detstion result [test_data]")
flags.DEFINE_string("test_result_dir", "test_result", "Directory name to save the anomaly test result [test_data/test_result]")
flags.DEFINE_boolean("crop", False, "True for training, False for testing [False]")
flags.DEFINE_integer("generate_test_images", 10, "Number of images to generate during test. [100]")
flags.DEFINE_boolean("ano_result_save", True, "True for save result of anomaly detection output")
flags.DEFINE_string('f', '', 'kernel')

FLAGS = flags.FLAGS

# # get the start time
# st = time.process_time()

with tf.Session(config=run_config) as sess:
    dcgan = DCGAN(
              sess,
              input_width = 128,
              input_height = 128,
              output_width = 128,
              output_height = 128,
              batch_size = 128,
              test_batch_size = FLAGS.test_batch_size,
              sample_num= 128,
              z_dim = 50,
              dataset_name = 'test_for_time',
              crop = FLAGS.crop,
              checkpoint_dir = FLAGS.checkpoint_dir,
              sample_dir= FLAGS.sample_dir,
              train_dir = FLAGS.train_dir,
              test_dir = FLAGS.test_dir)

    #show_all_variables()
    #dcgan.train(FLAGS)
    if not dcgan.load(FLAGS.checkpoint_dir)[0]:
        raise Exception("[!] Train a model first, then run test mode")
    
    dcgan.anomaly_detector(matching_layer_name='h3')
    n_test = len(dcgan.test_data_names)
    print('total number of test sample: ' + str(n_test))
    assert len(dcgan.test_data_names) > 0
    dcgan.train_anomaly_detector(FLAGS, dcgan.test_data_names)
    
# # get the end time
# et = time.process_time()

# # get execution time
# res = et - st
# print('CPU Execution time:', res, 'seconds')